# Decision Trees

### **Implementing Decision Trees for binary classification from scratch**

In [5]:
X = [
    [25, 0, 0, 15, 30],  # Age, has_car, spouse_car, commute_km, income
    [40, 1, 0, 3,  90],
    [35, 0, 1, 10, 85],
    [20, 0, 0, 30, 20],
    [50, 1, 1, 2, 110],
    [23, 0, 0, 25, 25],
]

y = [0, 1, 1, 0, 1, 0]  # 0 = No Buy, 1 = Buy

**Purity measure to find where to split the data**

Gini impurity: $G(D) = 1 - \sum{P_i^2}$ 

$P_i$ is fraction of that class

In [6]:
from collections import Counter


def gini(y):
    counts = Counter(y)
    impurity = 1.0
    n = len(y)
    for label in counts:
        prob_of_label = counts[label] / n
        impurity -= prob_of_label ** 2

    return impurity

**STEP 1. Try all possible splits to find the Best Feature**

1. loop over each feature
2. loop each unique value in the feature as threshold
3. split data based on the threshold
4. compute weighted gini impurity
5. keep the split that gives lowest impurity




In [7]:
def best_split(X, y):
    best_feature = None
    best_threshold = None
    best_gini = 1.0 # start with lowest impurity
    n_features = len(X[0])

    for feature_index in range(n_features):
        thresholds = set(row[feature_index] for row in X)
        for threshold in thresholds:
            left_y  = [y[i] for i in range(len(X)) if X[i][feature_index] <= threshold]
            right_y = [y[i] for i in range(len(X)) if X[i][feature_index] > threshold]

            if not left_y or not right_y: 
                continue # skip useless split
        
            gini_left = gini(left_y)
            gini_right = gini(right_y)
            weighted_gini = (len(left_y) * gini_left + len(right_y) * gini_right) / len(y)

            if (weighted_gini < best_gini):
                best_gini = weighted_gini
                best_feature = feature_index
                best_threshold = threshold

    return best_feature, best_threshold


**STEP 2: Recursively Build the Tree**

Once the best split is found: 
* Create a decision node with the feature and threshold 
* Recursively build left and right subtrees

Base Cases (when to stop recursion):
* All labels are the same → return a leaf with that label
* Max depth reached → return majority label
* No good split → return majority label

In [8]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value 

def build_tree(X, y, depth=0, max_depths=3):
    if (len(set(y)) == 1): # all same cases
        return Node(value=y[0])

    if depth >= max_depths:
        # majority class
        value = max(set(y), key=y.count)
        return Node(value=value)
    
    feature, threshold = best_split(X, y)
    
    if feature is None:
        value = max(set(y), key=y.count)
        return Node(value=value)
    
    # Split left and right indices
    left_indices = [i for i in range(len(X)) if X[i][feature] <= threshold]
    right_indices = [i for i in range(len(X)) if X[i][feature] > threshold]

    # Recursively build the left and right tree
    left = build_tree([X[i] for i in left_indices], [y[i] for i in left_indices], depth+1, max_depths)
    right = build_tree([X[i] for i in right_indices], [y[i] for i in right_indices], depth+1, max_depths)

    return Node(feature, threshold, left, right)

**STEP 3: Make Predictions Recursively**

In [9]:
def predict(tree, x):
    if tree.value is not None: # base case
        return tree.value
    if x[tree.feature] <= tree.threshold:
        return predict(tree.left, x)
    else:
        return predict(tree.right, x)

In [32]:
decision_tree = build_tree(X, y)
X_new = [40, 1, 0, 5, 40]
print(predict(decision_tree, X_new))

1


## Using Sklearn

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import datasets

In [5]:
X, y = datasets.make_moons(n_samples=1000, noise=0.2, random_state=42)
X.shape, y.shape

((1000, 2), (1000,))

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
X_train.shape

(800, 2)

In [24]:
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [26]:
X_new = [[293, 172]]
scaler = StandardScaler()
scaler.fit(X_train)
X_new_scaled = scaler.transform(X_new)
tree_clf.predict(X_new_scaled)

array([1], dtype=int64)

In [ ]:
# Use gridSearchCV to cross validate the dataset
